In [2]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline

In [2]:
!wget -O loan_train.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_train.csv


--2021-05-08 15:27:11--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_train.csv
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23101 (23K) [text/csv]
Saving to: ‘loan_train.csv’

loan_train.csv      100%[===================>]  22.56K  --.-KB/s    in 0s      

2021-05-08 15:27:18 (48.6 MB/s) - ‘loan_train.csv’ saved [23101/23101]



In [3]:
df = pd.read_csv('loan_train.csv')
df

,Unnamed: 0,Unnamed: 0.1,loan_status,Principal,terms,effective_date,due_date,age,education,Gender
0,0,0,PAIDOFF,1000,30,9/8/2016,10/7/2016,45,High School or Below,male
1,2,2,PAIDOFF,1000,30,9/8/2016,10/7/2016,33,Bechalor,female
2,3,3,PAIDOFF,1000,15,9/8/2016,9/22/2016,27,college,male
3,4,4,PAIDOFF,1000,30,9/9/2016,10/8/2016,28,college,female
4,6,6,PAIDOFF,1000,30,9/9/2016,10/8/2016,29,college,male
...,...,...,...,...,...,...,...,...,...,...
341,394,394,COLLECTION,800,15,9/11/2016,9/25/2016,32,High School or Below,male
342,395,395,COLLECTION,1000,30,9/11/2016,10/10/2016,25,High School or Below,male
343,397,397,COLLECTION,800,15,9/12/2016,9/26/2016,39,college,male
344,398,398,COLLECTION,1000,30,9/12/2016,11/10/2016,28,college,male


Change the date format and identify if the day is a weekend or a weekday

In [4]:
df['due_date'] = pd.to_datetime(df['due_date'])
df['effective_date'] = pd.to_datetime(df['effective_date'])



df['dayofweek'] = df['effective_date'].dt.dayofweek

df['weekend'] = df['dayofweek'].apply(lambda x:1 if (x>3) else 0)


df.head()


,Unnamed: 0,Unnamed: 0.1,loan_status,Principal,terms,effective_date,due_date,age,education,Gender,dayofweek,weekend
0,0,0,PAIDOFF,1000,30,2016-09-08,2016-10-07,45,High School or Below,male,3,0
1,2,2,PAIDOFF,1000,30,2016-09-08,2016-10-07,33,Bechalor,female,3,0
2,3,3,PAIDOFF,1000,15,2016-09-08,2016-09-22,27,college,male,3,0
3,4,4,PAIDOFF,1000,30,2016-09-09,2016-10-08,28,college,female,4,1
4,6,6,PAIDOFF,1000,30,2016-09-09,2016-10-08,29,college,male,4,1


identify wether a loan payee is a male or a female

In [5]:
df['Gender'].replace(to_replace=['male','female'],value=[1,0], inplace=True)

df.head()

,Unnamed: 0,Unnamed: 0.1,loan_status,Principal,terms,effective_date,due_date,age,education,Gender,dayofweek,weekend
0,0,0,PAIDOFF,1000,30,2016-09-08,2016-10-07,45,High School or Below,1,3,0
1,2,2,PAIDOFF,1000,30,2016-09-08,2016-10-07,33,Bechalor,0,3,0
2,3,3,PAIDOFF,1000,15,2016-09-08,2016-09-22,27,college,1,3,0
3,4,4,PAIDOFF,1000,30,2016-09-09,2016-10-08,28,college,0,4,1
4,6,6,PAIDOFF,1000,30,2016-09-09,2016-10-08,29,college,1,4,1


In [6]:
df[['Principal', 'terms', 'age', 'Gender', 'education']].head()


,Principal,terms,age,Gender,education
0,1000,30,45,1,High School or Below
1,1000,30,33,0,Bechalor
2,1000,15,27,1,college
3,1000,30,28,0,college
4,1000,30,29,1,college


In [7]:
Feature = df[['Principal','terms','age', 'Gender', 'weekend']]

Feature = pd.concat([Feature,pd.get_dummies(df['education'])], axis=1)
Feature.drop(['Master or Above'], axis = 1, inplace=True)
Feature.head()




,Principal,terms,age,Gender,weekend,Bechalor,High School or Below,college
0,1000,30,45,1,0,0,1,0
1,1000,30,33,0,0,1,0,0
2,1000,15,27,1,0,0,0,1
3,1000,30,28,0,1,0,0,1
4,1000,30,29,1,1,0,0,1


In [8]:
x = Feature

y = df['loan_status'].values

x = preprocessing.StandardScaler().fit(x).transform(x)

K-Nearest neighbour

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics


x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.25)

Ks = 15
mean_acc = np.zeros((Ks-1))
std_acc = np.zeros((Ks-1))
ConfustionMtx=[];


for n in range(1,Ks):
    neigh = KNeighborsClassifier(n_neighbors=n).fit(x_train,y_train)
    yhat = neigh.predict(x_test)
    mean_acc[n-1] = metrics.accuracy_score(y_test,yhat)
    std_acc[n-1]  = np.std(yhat==y_test)/np.sqrt(yhat.shape[0])
mean_acc

print("Best accuracy was achieved with", mean_acc.max(), "with k = ", mean_acc.argmax()+1)


Best accuracy was achieved with 0.7471264367816092 with k =  5


Everytime the script is run, the valueof max number of k changes, probable because of the changing test and training data sets 


.

Decision Tree

In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree



modeltree = DecisionTreeClassifier(criterion="entropy")
modeltree.fit(x_train, y_train)

predTree = modeltree.predict(x_test)
print(predTree[0:5])





['COLLECTION' 'COLLECTION' 'PAIDOFF' 'PAIDOFF' 'PAIDOFF']


.

Support Vector Machine

In [11]:

from sklearn import svm
from sklearn.metrics import f1_score


clsf = svm.SVC(kernel="sigmoid")
clsf.fit(x_train, y_train)

yhat = clsf.predict(x_test)

f1_score(y_test, yhat, average="weighted")



0.6652104411370473

.

logistic regression

In [13]:
from sklearn.linear_model import LogisticRegression


LR = LogisticRegression(C=0.01, solver="liblinear").fit(x_train,y_train)


yhat = LR.predict(x_test)

yhat_prob = LR.predict_proba(x_test)


from sklearn.metrics import jaccard_score
jaccard_score(y_test,yhat, pos_label = "PAIDOFF")




0.7471264367816092

Metric Evaluation

In [14]:
from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

In [20]:


knn_pred=neigh.predict(x)
jc1=jaccard_score(y, knn_pred, pos_label = "PAIDOFF")
fs1=f1_score(y, knn_pred, average='weighted')

tree_pred=modeltree.predict(x)
jc2=jaccard_score(y, tree_pred, pos_label = "PAIDOFF")
fs2=f1_score(y, tree_pred, average='weighted')

svm_pred=clsf.predict(x)
jc3=jaccard_score(y, svm_pred, pos_label = "PAIDOFF")
fs3=f1_score(y, svm_pred, average='weighted')

log_pred=LR.predict(x)
proba=LR.predict_proba(x)
jc4=jaccard_score(y, log_pred, pos_label = "PAIDOFF")
fs4=f1_score(y, log_pred, average='weighted')  
ll4=log_loss(y, proba)

list_jc = [jc1, jc2, jc3, jc4]
list_fs = [fs1, fs2, fs3, fs4]
list_ll = ['NA', 'NA', 'NA', ll4]


import pandas as pd

# fomulate the report format
df = pd.DataFrame(list_jc, index=['KNN','Decision Tree','SVM','Logistic Regression'])
df.columns = ['Jaccard']
df.insert(loc=1, column='F1-score', value=list_fs)
df.insert(loc=2, column='LogLoss', value=list_ll)
df.columns.name = 'Algorithm'
df

Algorithm,Jaccard,F1-score,LogLoss
KNN,0.737654,0.722773,NA
Decision Tree,0.807971,0.851931,NA
SVM,0.736232,0.642631,NA
Logistic Regression,0.753623,0.651584,0.562326
